In [ ]:
import pandas as pd
import numpy as np
import json

from os.path import exists, join
from matplotlib import pyplot as plt
from scipy.stats.mstats import mannwhitneyu
from scipy.stats import levene, probplot, shapiro

In [ ]:
with open('../config.json') as f:
        config = json.load(f)

info_path = config['publons_info']
if not exists(info_path):
    print('Nenhum dado publons_info a ser analisado.')
    
info_original_file_path = join(info_path, 'publons_info.csv')
if not exists(info_original_file_path):
    print('Nenhum dado publons_info a ser analisado.')

info_file_path = join(info_path, 'publons_info_unique.csv')
if not exists(info_file_path):
    print('Nenhum dado publons_info_unique a ser analisado.')
    
info_file_filtered_path = join(info_path, 'publons_info_unique_filtered.csv')
if not exists(info_file_filtered_path):
    print('Nenhum dado publons_info_unique_filtered a ser analisado.')
    
results_path = config['publons_results']
if not exists(results_path):
    print('Nenhum dado publons_results a ser analisado.')
    
results_original_file_path = join(results_path, 'results_publons.csv')
if not exists(results_original_file_path):
    print('Nenhum dado results_publons a ser analisado.')
    
results_gender_path = join(results_path, 'results_publons_gender.csv')
if not exists(results_gender_path):
    print('Nenhum dado results_publons_gender a ser analisado.')
    
results_time_path = join(results_path, 'results_publons_gender_time.csv')
if not exists(results_time_path):
    print('Nenhum dado results_publons_gender_time a ser analisado.')        
    
results_file_path = join(results_path, 'results_publons_gender_time_anonymized.csv')
if not exists(results_file_path):
    print('Nenhum dado results_publons_gender_time_anonymized a ser analisado.')

# Estatísticas Básicas da Coleta de Dados

In [ ]:
df_info_original = pd.read_csv(info_original_file_path)
df_info_original_size = len(df_info_original.index)
print('Docentes da USP: ', df_info_original_size)

df_info = pd.read_csv(info_file_path)
df_info_size = len(df_info.index)
print('Ids Publons existentes e únicos: ', df_info_size)
print('Docentes sem ids Publons ou com mais de um registro na USP: ', df_info_original_size - df_info_size)

df_info_filtered = pd.read_csv(info_file_filtered_path)
df_info_filtered_size = len(df_info_filtered.index)
print('Ids Publons filtrados: ', df_info_filtered_size)
print('Inconsistência de identidade entre DataUSP e Publons: ', df_info_size - df_info_filtered_size)

df_results = pd.read_csv(results_original_file_path)
df_results_size = len(df_results.index)
print('Dados Publons recuperados: ', df_results_size)
print('Docentes com dados Publons não baixados: ', df_info_filtered_size - df_results_size )

df_gender = pd.read_csv(results_gender_path)
df_gender_size = len(df_gender.index)
print('Docentes com gênero encontrado: ', df_gender_size)
print('Docentes com informação de gênero não encontrada: ', df_results_size - df_gender_size )

df_time = pd.read_csv(results_time_path)
df_time_size = len(df_time.index)
print('Docentes com tempo de trabalho encontrado: ', df_time_size)
print('Docentes com informação de tempo de trabalho não encontrada: ', df_gender_size - df_time_size)

# Análise do Resultado Final


## Estatística Descritiva

In [ ]:
# Banco utilizado para as análises posteriores
df = pd.read_csv(results_file_path)
df.fillna(0, inplace=True)
# Verifica quantos pesquisadores não possuem citações
rows_to_drop = df.loc[(df['citations'] == 0) | (df['h_index'] == 0)].index
print('Docentes sem citacões ou h_index: ', len(rows_to_drop))
df = df.drop(rows_to_drop, axis=0)
df_size = len(df.index)
print('Banco final: ', df_size)

In [ ]:
all_people = df_size
df['gender'].value_counts()
men = df['gender'].value_counts()['M']
percentage_men = men/all_people*100

women = df['gender'].value_counts()['F']
percentage_women = women/all_people*100

print('Total de Docentes: ', all_people)
print('Homens: {} ({} % de {})'.format(men, round(percentage_men,2), all_people))
print('Mulheres: {} ({} % de {})'.format(women, round(percentage_women,2), all_people))

In [ ]:
print('Quantidade total com 9 ou menos anos de trabalho: ', len(df.loc[df['time'] < 10].index))
print('Quantidade total com 10 a 19 anos de trabalho: ', len(df.loc[(df['time'] < 20) & (df['time'] >= 10)].index))
print('Quantidade total com mais 20 ou mais anos de trabalho: ', len(df.loc[df['time'] >= 20].index))

In [ ]:
men_9 = len(df.loc[(df['time'] < 10) & (df['gender'] == 'M')].index)
men_10_19 = len(df.loc[(df['time'] < 20) & (df['time'] >= 10) & (df['gender'] == 'M')].index)
men_20 = len(df.loc[(df['time'] >= 20) & (df['gender'] == 'M')].index)

print('Total de Docentes: ', all_people)
print('Homens: {} ({} % de {})'.format(men, round(percentage_men,2), all_people))
print('Quantidade de homens com 9 ou menos anos de trabalho: {} ({} porcentagem de {})'
      .format(men_9, round(men_9/men*100,2), men))
print('Quantidade de homens com 10 a 19 anos de trabalho: {} ({} porcentagem de {})'
      .format(men_10_19, round(men_10_19/men*100,2), men))
print('Quantidade de homens com 20 ou mais anos de trabalho: {} ({} porcentagem de {})'
      .format(men_20, round(men_20/men*100,2), men))

In [ ]:
women_9 = len(df.loc[(df['time'] < 10) & (df['gender'] == 'F')].index)
women_10_19 = len(df.loc[(df['time'] < 20) & (df['time'] >= 10) & (df['gender'] == 'F')].index)
women_20 = len(df.loc[(df['time'] >= 20) & (df['gender'] == 'F')].index)

print('Total de Docentes: ', all_people)
print('Mulheres: {} ({} % de {})'.format(women, round(percentage_women,2), all_people))
print('Quantidade de mulheres com 9 ou menos anos de trabalho: {} ({} porcentagem de {})'
      .format(women_9, round(women_9/women*100,2), women))
print('Quantidade de mulheres com 10 a 19 anos de trabalho: {} ({} porcentagem de {})'
      .format(women_10_19, round(women_10_19/women*100,2), women))
print('Quantidade de mulheres com mais de 20 anos de trabalho: {} ({} porcentagem de {})'
      .format(women_20, round(women_20/women*100,2), women))

# Exploração dos Dados e Verificação da Distribuição

In [ ]:
df.head(2)

In [ ]:
df.describe()

In [ ]:
df[['wos_publications','citations','citations_per_item','citations_per_year','h_index','time']].hist(figsize=(12,10))

In [ ]:
df['gender'].value_counts().plot(kind='bar')

## Verificação da não normalidade nos dados com Q-Q Plots e Shapiro Test
alpha < 0.01

In [ ]:
probplot(df['wos_publications'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Publications")
plt.show()
probplot(df['citations'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Citations")
plt.show()
probplot(df['citations'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - H-Index")
plt.show()

In [ ]:
print('Análise de normalidade nos dados: ')
print('Publicações')
print(shapiro(df['wos_publications']))
print('Citações')
print(shapiro(df['citations']))
print('Citações por Item')
print(shapiro(df['citations_per_item']))
print('Citações por Ano')
print(shapiro(df['citations_per_year']))
print('H-Index')
print(shapiro(df['h_index']))

#### Os resultados claramente apontam para uma distribuição não normal.

## Check for Lognormality
https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0146762

"For institutions, it seems to fit roughly to a log-normal function. The largest citations, however, deviate from the lognormal fit, and follow a power law decay."

In [ ]:
df['index'] = df.index
df['wos_publications_log'] = np.log(df.wos_publications)
df['citations_log'] = np.log(df.citations)
df['citations_per_item_log'] = np.log(df.citations_per_item)
df['citations_per_year_log'] = np.log(df.citations_per_year)
df['h_index_log'] = np.log(df.h_index)

### Histogramas

In [ ]:
df['wos_publications_log'].hist(figsize=(5,5))
plt.show()
df['citations_log'].hist(figsize=(5,5))
plt.show()
df['citations_per_item_log'].hist(figsize=(5,5))
plt.show()
df['citations_per_year_log'].hist(figsize=(5,5))
plt.show()
df['h_index_log'].hist(figsize=(5,5))
plt.show()

### Scatter Plots (Verificação de Outliers)

In [ ]:
df.plot.scatter(x='index', y='wos_publications_log', figsize=(10,5))
df.plot.scatter(x='index', y='citations_log', figsize=(10,5))
df.plot.scatter(x='index', y='citations_per_item_log', figsize=(10,5))
df.plot.scatter(x='index', y='citations_per_year_log', figsize=(10,5))
df.plot.scatter(x='index', y='h_index_log', figsize=(10,5))

### Análise de normalidade nos dados logarítimicos (Q-Q Plot):
https://pdfs.semanticscholar.org/f2b9/98d29492e9b0a234bf13adc3fa8c9ae6ab32.pdf

In [ ]:
probplot(df['wos_publications_log'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Publications")
plt.show()
probplot(df['citations_log'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - Citations")
plt.show()
probplot(df['citations_log'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot - H-Index")
plt.show()

### Análise de normalidade nos dados logarítimicos (Shapiro Test):
a < 0.01

In [ ]:
print('Publicações')
print(shapiro(df['wos_publications_log']))
print('Citações')
print(shapiro(df['citations_log']))
print('Citações por Item')
print(shapiro(df['citations_per_item_log']))
print('Citações por Ano')
print(shapiro(df['citations_per_year_log']))
print('H-Index')
print(shapiro(df['h_index_log']))

#### Os dados indicam que mesmo com os dados transformados para escala log natural, eles ainda não seguem uma distribuição normal.

# Análise Por Gênero

## Estatística Descritiva

In [ ]:
display(df.groupby(by='gender')['wos_publications'].describe())
display(df.groupby(by='gender')['wos_publications'].var())
display(df.groupby(by='gender')['wos_publications_log'].var())
print('Citações')
display(df.groupby(by='gender').describe()['citations'])
display(df.groupby(by='gender')['citations'].var())
display(df.groupby(by='gender')['citations_log'].var())
print('Citações por item')
display(df.groupby(by='gender').describe()['citations_per_item'])
display(df.groupby(by='gender')['citations_per_item'].var())
display(df.groupby(by='gender')['citations_per_item_log'].var())
print('Citações por ano')
display(df.groupby(by='gender').describe()['citations_per_year'])
display(df.groupby(by='gender')['citations_per_year'].var())
display(df.groupby(by='gender')['citations_per_year_log'].var())
print('H-Index')
display(df.groupby(by='gender').describe()['h_index'])
display(df.groupby(by='gender')['h_index'].var())
display(df.groupby(by='gender')['h_index_log'].var())

In [ ]:
men = df.loc[df['gender'] == 'M']
women = df.loc[df['gender'] == 'F']

## Publicações, Citações e H-Index por Gênero e Tempo de Trabalho

In [ ]:
men.groupby('time')['wos_publications'].mean().plot(figsize=(10,5))
women.groupby('time')['wos_publications'].mean().plot()
plt.legend(['M','F'])
plt.title('Publicações por Gênero e Tempo de Trabalho')
plt.show()

men.groupby('time')['citations'].mean().plot(figsize=(10,5))
women.groupby('time')['citations'].mean().plot()
plt.legend(['M','F'])
plt.title('Citações por Gênero e Tempo de Trabalho')
plt.show()

men.groupby('time')['citations_per_item'].mean().plot(figsize=(10,5))
women.groupby('time')['citations_per_item'].mean().plot()
plt.legend(['M','F'])
plt.title('Citações por Item por Gênero e Tempo de Trabalho')
plt.show()

men.groupby('time')['citations_per_year'].mean().plot(figsize=(10,5))
women.groupby('time')['citations_per_year'].mean().plot()
plt.legend(['M','F'])
plt.title('Citações por Ano por Gênero e Tempo de Trabalho')
plt.show()

men.groupby('time')['h_index'].mean().plot(figsize=(10,5))
women.groupby('time')['h_index'].mean().plot()
plt.legend(['M','F'])
plt.title('H-Index por Gênero e Tempo de Trabalho')
plt.show()

## Histogramas de cada gênero Lognormal

In [ ]:
women['wos_publications_log'].hist(figsize=(5,5))
plt.show()
men['citations_log'].hist(figsize=(5,5))
women['citations_log'].hist(figsize=(5,5))
plt.show()
men['citations_per_item_log'].hist(figsize=(5,5))
women['citations_per_item_log'].hist(figsize=(5,5))
plt.show()
men['citations_per_year_log'].hist(figsize=(5,5))
women['citations_per_year_log'].hist(figsize=(5,5))
plt.show()
men['h_index_log'].hist(figsize=(5,5))
women['h_index_log'].hist(figsize=(5,5))

## Boxplots

In [ ]:
df[['gender','wos_publications_log']].groupby(by='gender').boxplot(figsize=(10,5))
df[['gender','citations_log']].groupby(by='gender').boxplot(figsize=(10,5))
df[['gender','citations_per_year_log']].groupby(by='gender').boxplot(figsize=(10,5))
df[['gender','citations_per_item_log']].groupby(by='gender').boxplot(figsize=(10,5))

## Análise de normalidade nos dados logarítimicos
alpha < 0.01

In [ ]:
print('Homens')
print('Publicações')
print(shapiro(men['wos_publications_log']))
print('Citações')
print(shapiro(men['citations_log']))
print('Citações por Item')
print(shapiro(men['citations_per_item_log']))
print('Citações por Ano')
print(shapiro(men['citations_per_year_log']))
print('H-Index')
print(shapiro(men['h_index_log']))

print('\nMulheres')
print('Publicações')
print(shapiro(women['wos_publications_log']))
print('Citações')
print(shapiro(women['citations_log']))
print('Citações por Item')
print(shapiro(women['citations_per_item_log']))
print('Citações por Ano')
print(shapiro(women['citations_per_year_log']))
print('H-Index')
print(shapiro(women['h_index_log']))

## Teste Estatístico
### Análise de diferença de variância
alpha < 0.01

In [ ]:
print('Publicações')
print(levene(men['wos_publications_log'], women['wos_publications_log']))
print('Citações')
print(levene(men['citations_log'], women['citations_log']))
print('Citações por Item')
print(levene(men['citations_per_item_log'], women['citations_per_item_log']))
print('Citações por Ano')
print(levene(men['citations_per_year_log'], women['citations_per_year_log']))
print('H-Index')
print(levene(men['h_index_log'], women['h_index_log']))

### Uso do Mann Whitney Test 
#### Já que os dados não são normalmente distribuídos e há diferenças na variância em alguns caso.

alpha < 0.01

In [ ]:
print('Dados na escala Log \n')
test_wos_log = mannwhitneyu(men['wos_publications_log'], women['wos_publications_log'])
test_citations_log = mannwhitneyu(men['citations_log'], women['citations_log'])
test_citations_per_item_log = mannwhitneyu(men['citations_per_item_log'], women['citations_per_item_log'])
test_citations_per_year_log = mannwhitneyu(men['citations_per_year_log'], women['citations_per_year_log'])
test_h_index_log = mannwhitneyu(men['h_index_log'], women['h_index_log'])

print('Publicações')
print(test_wos_log)
print('Citações')
print(test_citations_log)
print('Citações por item')
print(test_citations_per_item_log)
print('Citações por ano')
print(test_citations_per_year_log)
print('H-index')
print(test_h_index_log)

# Por Gênero e Tempo de Trabalho

## Estatística Descritiva

In [ ]:
years_9 = df.loc[(df['time'] < 10)]
years_10_19 = df.loc[(df['time'] < 20) & (df['time'] >= 10)]
years_20 = df.loc[(df['time'] >= 20)]

In [ ]:
print('Publicações')
display(years_9.groupby(by='gender').describe()['wos_publications'])
print('Citações')
display(years_9.groupby(by='gender').describe()['citations'])
print('Citações por item')
display(years_9.groupby(by='gender').describe()['citations_per_item'])
print('Citações por ano')
display(years_9.groupby(by='gender').describe()['citations_per_year'])
print('H-Index')
display(years_9.groupby(by='gender').describe()['h_index'])

In [ ]:
print('Publicações')
display(years_10_19.groupby(by='gender').describe()['wos_publications'])
print('Citações')
display(years_10_19.groupby(by='gender').describe()['citations'])
print('Citações por item')
display(years_10_19.groupby(by='gender').describe()['citations_per_item'])
print('Citações por ano')
display(years_10_19.groupby(by='gender').describe()['citations_per_year'])
print('H-Index')
display(years_10_19.groupby(by='gender').describe()['h_index'])

In [ ]:
print('Publicações')
display(years_20.groupby(by='gender').describe()['wos_publications'])
print('Citações')
display(years_20.groupby(by='gender').describe()['citations'])
print('Citações por item')
display(years_20.groupby(by='gender').describe()['citations_per_item'])
print('Citações por ano')
display(years_20.groupby(by='gender').describe()['citations_per_year'])
print('H-Index')
display(years_20.groupby(by='gender').describe()['h_index'])

## Boxplots

In [ ]:
years_9[['gender','wos_publications_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_9[['gender','citations_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_9[['gender','citations_per_item_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_9[['gender','citations_per_year_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_9[['gender','h_index_log']].groupby(by='gender').boxplot(figsize=(10,5))

In [ ]:
years_10_19[['gender','wos_publications_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_10_19[['gender','citations_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_10_19[['gender','citations_per_item_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_10_19[['gender','citations_per_year_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_10_19[['gender','h_index_log']].groupby(by='gender').boxplot(figsize=(10,5))

In [ ]:
years_20[['gender','wos_publications_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_20[['gender','citations_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_20[['gender','citations_per_item_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_20[['gender','citations_per_year_log']].groupby(by='gender').boxplot(figsize=(10,5))
years_20[['gender','h_index_log']].groupby(by='gender').boxplot(figsize=(10,5))

## Testes Estatísticos
alpha < 0.01

### 0-10 Anos de trabalho

In [ ]:
men = years_9.loc[(years_9['gender'] == 'M')]
women = years_9.loc[(years_9['gender'] == 'F')]

men['wos_publications_log'].hist(figsize=(5,5))
women['wos_publications_log'].hist(figsize=(5,5))
plt.show()
men['citations_log'].hist(figsize=(5,5))
women['citations_log'].hist(figsize=(5,5))
plt.show()
men['citations_per_item_log'].hist(figsize=(5,5))
women['citations_per_item_log'].hist(figsize=(5,5))
plt.show()
men['citations_per_year_log'].hist(figsize=(5,5))
women['citations_per_year_log'].hist(figsize=(5,5))
plt.show()
men['h_index_log'].hist(figsize=(5,5))
women['h_index_log'].hist(figsize=(5,5))
plt.show()

print('Análise de normalidade (Shapiro Test)')
print('\nHomens')
print('Publicações')
print(shapiro(men['wos_publications_log']))
print('Citações')
print(shapiro(men['citations_log']))
print('Citações por Item')
print(shapiro(men['citations_per_item_log']))
print('Citações por Ano')
print(shapiro(men['citations_per_year_log']))
print('H-Index')
print(shapiro(men['h_index_log']))

print('\nMulheres')
print('Publicações')
print(shapiro(women['wos_publications_log']))
print('Citações')
print(shapiro(women['citations_log']))
print('Citações por Item')
print(shapiro(women['citations_per_item_log']))
print('Citações por Ano')
print(shapiro(women['citations_per_year_log']))
print('H-Index')
print(shapiro(women['h_index_log']))

print('\nAnálise de diferença de variância (Levene)')
print('Publicações')
print(levene(men['wos_publications_log'], women['wos_publications_log']))
print('Citações')
print(levene(men['citations_log'], women['citations_log']))
print('Citações por Item')
print(levene(men['citations_per_item_log'], women['citations_per_item_log']))
print('Citações por Ano')
print(levene(men['citations_per_year_log'], women['citations_per_year_log']))
print('H-Index')
print(levene(men['h_index_log'], women['h_index_log']))

print('\n')
print('Mann Whitney U')
test_wos_log = mannwhitneyu(men['wos_publications_log'], women['wos_publications_log'])
test_citations_log = mannwhitneyu(men['citations_log'], women['citations_log'])
test_citations_per_item_log = mannwhitneyu(men['citations_per_item_log'], women['citations_per_item_log'])
test_citations_per_year_log = mannwhitneyu(men['citations_per_year_log'], women['citations_per_year_log'])
test_h_index_log = mannwhitneyu(men['h_index_log'], women['h_index_log'])

print('Publicações')
print(test_wos_log)
print('Citações')
print(test_citations_log)
print('Citações por item')
print(test_citations_per_item_log)
print('Citações por ano')
print(test_citations_per_year_log)
print('H-index')
print(test_h_index_log)

### 10-19 Anos de Trabalho

In [ ]:
men = years_10_19.loc[(years_10_19['gender'] == 'M')]
women = years_10_19.loc[(years_10_19['gender'] == 'F')]

men['wos_publications_log'].hist(figsize=(5,5))
women['wos_publications_log'].hist(figsize=(5,5))
plt.show()
men['citations_log'].hist(figsize=(5,5))
women['citations_log'].hist(figsize=(5,5))
plt.show()
men['citations_per_item_log'].hist(figsize=(5,5))
women['citations_per_item_log'].hist(figsize=(5,5))
plt.show()
men['citations_per_year_log'].hist(figsize=(5,5))
women['citations_per_year_log'].hist(figsize=(5,5))
plt.show()
men['h_index_log'].hist(figsize=(5,5))
women['h_index_log'].hist(figsize=(5,5))
plt.show()

print('Análise de normalidade (Shapiro Test)')
print('\nHomens')
print('Publicações')
print(shapiro(men['wos_publications_log']))
print('Citações')
print(shapiro(men['citations_log']))
print('Citações por Item')
print(shapiro(men['citations_per_item_log']))
print('Citações por Ano')
print(shapiro(men['citations_per_year_log']))
print('H-Index')
print(shapiro(men['h_index_log']))

print('\nMulheres')
print('Publicações')
print(shapiro(women['wos_publications_log']))
print('Citações')
print(shapiro(women['citations_log']))
print('Citações por Item')
print(shapiro(women['citations_per_item_log']))
print('Citações por Ano')
print(shapiro(women['citations_per_year_log']))
print('H-Index')
print(shapiro(women['h_index_log']))

print('\nAnálise de diferença de variância (Levene)')
print('Publicações')
print(levene(men['wos_publications_log'], women['wos_publications_log']))
print('Citações')
print(levene(men['citations_log'], women['citations_log']))
print('Citações por Item')
print(levene(men['citations_per_item_log'], women['citations_per_item_log']))
print('Citações por Ano')
print(levene(men['citations_per_year_log'], women['citations_per_year_log']))
print('H-Index')
print(levene(men['h_index_log'], women['h_index_log']))

print('\n')
print('Mann Whitney U')
test_wos_log = mannwhitneyu(men['wos_publications_log'], women['wos_publications_log'])
test_citations_log = mannwhitneyu(men['citations_log'], women['citations_log'])
test_citations_per_item_log = mannwhitneyu(men['citations_per_item_log'], women['citations_per_item_log'])
test_citations_per_year_log = mannwhitneyu(men['citations_per_year_log'], women['citations_per_year_log'])
test_h_index_log = mannwhitneyu(men['h_index_log'], women['h_index_log'])

print('Publicações')
print(test_wos_log)
print('Citações')
print(test_citations_log)
print('Citações por item')
print(test_citations_per_item_log)
print('Citações por ano')
print(test_citations_per_year_log)
print('H-index')
print(test_h_index_log)

### 20 ou mais Anos de Trabalho

In [ ]:
men = years_20.loc[(years_20['gender'] == 'M')]
women = years_20.loc[(years_20['gender'] == 'F')]

men['wos_publications_log'].hist(figsize=(5,5))
women['wos_publications_log'].hist(figsize=(5,5))
plt.show()
men['citations_log'].hist(figsize=(5,5))
women['citations_log'].hist(figsize=(5,5))
plt.show()
men['citations_per_item_log'].hist(figsize=(5,5))
women['citations_per_item_log'].hist(figsize=(5,5))
plt.show()
men['citations_per_year_log'].hist(figsize=(5,5))
women['citations_per_year_log'].hist(figsize=(5,5))
plt.show()
men['h_index_log'].hist(figsize=(5,5))
women['h_index_log'].hist(figsize=(5,5))
plt.show()

print('Análise de normalidade (Shapiro Test)')
print('\nHomens')
print('Publicações')
print(shapiro(men['wos_publications_log']))
print('Citações')
print(shapiro(men['citations_log']))
print('Citações por Item')
print(shapiro(men['citations_per_item_log']))
print('Citações por Ano')
print(shapiro(men['citations_per_year_log']))
print('H-Index')
print(shapiro(men['h_index_log']))

print('\nMulheres')
print('Publicações')
print(shapiro(women['wos_publications_log']))
print('Citações')
print(shapiro(women['citations_log']))
print('Citações por Item')
print(shapiro(women['citations_per_item_log']))
print('Citações por Ano')
print(shapiro(women['citations_per_year_log']))
print('H-Index')
print(shapiro(women['h_index_log']))

print('\nAnálise de diferença de variância (Levene)')
print('Publicações')
print(levene(men['wos_publications_log'], women['wos_publications_log']))
print('Citações')
print(levene(men['citations_log'], women['citations_log']))
print('Citações por Item')
print(levene(men['citations_per_item_log'], women['citations_per_item_log']))
print('Citações por Ano')
print(levene(men['citations_per_year_log'], women['citations_per_year_log']))
print('H-Index')
print(levene(men['h_index_log'], women['h_index_log']))

print('\n')
print('Mann Whitney U')
test_wos_log = mannwhitneyu(men['wos_publications_log'], women['wos_publications_log'])
test_citations_log = mannwhitneyu(men['citations_log'], women['citations_log'])
test_citations_per_item_log = mannwhitneyu(men['citations_per_item_log'], women['citations_per_item_log'])
test_citations_per_year_log = mannwhitneyu(men['citations_per_year_log'], women['citations_per_year_log'])
test_h_index_log = mannwhitneyu(men['h_index_log'], women['h_index_log'])

print('Publicações')
print(test_wos_log)
print('Citações')
print(test_citations_log)
print('Citações por item')
print(test_citations_per_item_log)
print('Citações por ano')
print(test_citations_per_year_log)
print('H-index')
print(test_h_index_log)